# Playbill AI

# Use Case

Gen AI model (Gemini 2.0 flash) is used as a part of the service for aggregation of theater playbills.

## Problem

There are a lot of theaters in Moscow and tracking premieres in all of them is a unpleasant task. Despite there are aggregation services that collect this info, they are mostly uncomfortable for users as they don't have different filters, such as choosing location, specific play or stage director. Moreover, crawling through a variety of filters is also time-consuming and discouraging for users.

## Solution

The provided solution allows to transform arbitrary user input into an appropriate SQL-query and then return the response basing on the data in the database. Currently only text is supported, but it's possible to extend it to voice input as well.

## Example

E.g. user asks: "Plays for the coming month by Shakespeare, Williams or Chekhov" that is transformed into SQL-query that is passed into database. The result is "There are no plays by Shakespeare, Williams, or Chekhov in the coming month" because there aren't any performances by these authors in the local database. See more examples below. 


# Gen AI Capabilities

The project combines such AI capabilities as
- Structured output (SQL-query)
- Few-show prompting
- Function Calling
- Grounding (for showing reviews)

# Future

The developed solution will be used in conjunction with developing API for theatres (link will be provided later) as a part of aggregation service deployed both as a web-application and as telegram bot

# Implementation
## Setup

Start by installing and importing the Python SDK.

In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [2]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Automated retry

In [3]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry
from google import genai
from google.genai import types

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Create a local database

For this minimal example, we'll create a local SQLite database and add some synthetic data to have something to query.

Load the `sql` IPython extension so you can interact with the database using magic commands (the `%` instructions) to create a new, empty SQLite database.

In [4]:
TEST_MODE = False  # Set to False if you want to persist

%load_ext sql
if TEST_MODE:
    %sql sqlite:///:memory:
else:
    %sql sqlite:///sample.db

Create the tables and insert some data (later it'll be provided by service via API).

In [5]:
import sqlite3

if TEST_MODE:
    db_conn = sqlite3.connect(':memory:')  # In-memory for testing
else:
    db_file = "sample.db"
    db_conn = sqlite3.connect(db_file)     # Persistent on disk

In [6]:
%%sql
CREATE TABLE theaters (
    theater_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    acronym TEXT
);

CREATE TABLE stages (
    stage_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    address TEXT,
    theater_id INTEGER,
    FOREIGN KEY (theater_id) REFERENCES theaters(theater_id)
);

CREATE TABLE performances (
    performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    stage_id INTEGER,
    datetime TEXT NOT NULL,
    director TEXT,
    author TEXT,
    FOREIGN KEY (stage_id) REFERENCES stages(stage_id)
);


 * sqlite:///sample.db
Done.
Done.
Done.


[]

In [7]:
%%sql
INSERT INTO theaters (name, acronym) VALUES
  ('Российский академический молодежный театр', 'РАМТ'),
  ('Мастерская Петра Фоменко', 'Фоменко'),
  ('Театр Сатирикон имени Аркадия Райкина', 'Сатирикон');

INSERT INTO stages (name, address, theater_id) VALUES
  ('Большая сцена', 'Театральная пл., д. 2, Москва', 1),
  ('Маленькая сцена', 'Театральная пл., д. 2, Москва', 1),
  ('Новая сцена', 'Кутузовский пр-т, д. 30, Москва', 2),
  ('Старая сцена', 'Кутузовский пр-т, д. 30, Москва', 2),
  ('Дворец на Яузе', 'ул. Журавлёва, д. 1, Москва', 3),
  ('Центральный дом актера', 'Арбат, д. 35, Москва', 3);

INSERT INTO performances (title, stage_id, datetime, director, author) VALUES
  ('Женщины Лазаря', 1, '2025-04-16T19:00:00', 'Алексей Бородин', 'Марина Степнова'),
  ('Пустые поезда', 2, '2025-04-30T16:00:00', 'Дмитрий Данилов', 'Дмитрий Данилов'),
  ('Чайка', 3, '2025-04-27T19:00:00', 'Петр Фоменко', 'Антон Чехов'),
  ('Светлые души, или О том, как написать рассказ', 4, '2025-04-28T19:00:00', 'Евгений Каменькович', 'Василий Шукшин'),
  ('Гроза', 5, '2025-04-20T19:00:00', 'Константин Райкин', 'Александр Островский'),
  ('Дама с собачкой', 6, '2025-04-19T19:00:00', 'Константин Райкин', 'Антон Чехов');



 * sqlite:///sample.db
3 rows affected.
6 rows affected.
6 rows affected.


[]

## Define database functions


In [8]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['theaters', 'sqlite_sequence', 'stages', 'performances']

In [9]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


[describe_table(table) for table in list_tables()]

 - DB CALL: list_tables()
 - DB CALL: describe_table(theaters)
 - DB CALL: describe_table(sqlite_sequence)
 - DB CALL: describe_table(stages)
 - DB CALL: describe_table(performances)


[[('theater_id', 'INTEGER'), ('name', 'TEXT'), ('acronym', 'TEXT')],
 [('name', ''), ('seq', '')],
 [('stage_id', 'INTEGER'),
  ('name', 'TEXT'),
  ('address', 'TEXT'),
  ('theater_id', 'INTEGER')],
 [('performance_id', 'INTEGER'),
  ('title', 'TEXT'),
  ('stage_id', 'INTEGER'),
  ('datetime', 'TEXT'),
  ('director', 'TEXT'),
  ('author', 'TEXT')]]

In [10]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from performances limit 3")

 - DB CALL: execute_query(select * from performances limit 3)


[(1,
  'Женщины Лазаря',
  1,
  '2025-04-16T19:00:00',
  'Алексей Бородин',
  'Марина Степнова'),
 (2,
  'Пустые поезда',
  2,
  '2025-04-30T16:00:00',
  'Дмитрий Данилов',
  'Дмитрий Данилов'),
 (3, 'Чайка', 3, '2025-04-27T19:00:00', 'Петр Фоменко', 'Антон Чехов')]

In [11]:
def get_available_values(table_name: str, field_name: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    sql = f"select {field_name} from {table_name}"
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


get_available_values(table_name="theaters", field_name="name")

 - DB CALL: execute_query(select name from theaters)


[('Российский академический молодежный театр',),
 ('Мастерская Петра Фоменко',),
 ('Театр Сатирикон имени Аркадия Райкина',)]

In [12]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query, get_available_values]

instruction = """You are a helpful chatbot that can interact with an SQL database
for theater playbills. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, get_available_values to get values in the specific field of a table and
execute_query to issue an SQL SELECT query. Usually, you need to execute functions
in the specified order. Always check what are the valid values for the fields that you're gonna query

Example: 
    Question: 
        Спектакли в РАМТе после 17 апреля под авторством Данилова
    Your actions:
        1. list_tables() to get available tables: 'theaters', 'stages', 'performances'
        2. describe_table(performances) to get fields of performance table"
        3. describe_table(theaters) to get fields of theaters table
        4. describe_table(stages) to get fields of stages table
        5. get_available_values(table_name="performances", field_name="author") to get list of authors and find value for "Данилов"
        6. get_available_values(table_name="theaters", field_name="name") to get list of names and find value for "РАМТ"
        7. execute_query(...) to get appropriate result
"""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [13]:
response = chat.send_message('Спектакли Данилова в РАМТЕ в апреле 2025')
print(f"\n{response.text}")

 - DB CALL: list_tables()
 - DB CALL: describe_table(performances)
 - DB CALL: describe_table(theaters)
 - DB CALL: describe_table(stages)
 - DB CALL: execute_query(select author from performances)
 - DB CALL: execute_query(select name from theaters)
 - DB CALL: execute_query(SELECT T1.title FROM performances AS T1 JOIN stages AS T2 ON T1.stage_id = T2.stage_id JOIN theaters AS T3 ON T2.theater_id = T3.theater_id WHERE T3.name = 'Российский академический молодежный театр' AND T1.author = 'Дмитрий Данилов' AND T1.datetime LIKE '2025-04%';)

В апреле 2025 года в РАМТе идет спектакль Данилова "Пустые поезда".


It's possible to extract the created SQL query to use it in a more traditional way (e.g. to show the user the table with all options). This is also useful to check that there are no hallucinations in the output.

In [14]:
def get_sql_queries(chat):
    """Get all SQL queries in the chat history."""
    sql_queries = []
    for event in chat.get_history():
        if function_call := event.parts[0].function_call:
            if sql_arg := function_call.args.get('sql'):
                sql_queries.append(sql_arg)
    return sql_queries

get_sql_queries(chat)

["SELECT T1.title FROM performances AS T1 JOIN stages AS T2 ON T1.stage_id = T2.stage_id JOIN theaters AS T3 ON T2.theater_id = T3.theater_id WHERE T3.name = 'Российский академический молодежный театр' AND T1.author = 'Дмитрий Данилов' AND T1.datetime LIKE '2025-04%';"]

## Getting reviews using Google search grounding (in development)

In [15]:
from IPython.display import Markdown

review_instruction = f"""Get reviews of the mentioned performance.
Do not mention any unnecessary information, only reviews."""

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=review_instruction,
        tools=[types.Tool(google_search=types.GoogleSearch())],
    ),
)


review_response = chat.send_message(response.text)
rc = review_response.candidates[0]
Markdown(rc.content.parts[0].text)

Вот некоторые отзывы о спектакле Данилова «Пустые поезда» в РАМТе:

*   «Спектакль оказался приятно неожиданным. Текст действительно абсолютно несценический. Там нет ни диалогов, ни яркого внешнего действия, ни какого бы то ни было связного сюжета. Тем не менее вышел, на мой взгляд, замечательный спектакль».
*   «Постановка для Маленькой сцены театра основана на цикле очерков «Пустые поезда 2022 года» известного современного писателя и драматурга».
*   «Смешанное чувство ностальгии и отвращения, ведь эти поезда славятся отнюдь не романтическими настроениями».
*   «Каждая поездка – это приключение, большое или маленькое, «медитативный туризм», когда за окном пролетают пейзажи, восходы и закаты. Через весь сюжет звучит история потери в годы пандемии матери и каждая зарисовка служит еще одним штрихом в большой картине воспоминаний».
*   «Поездки сопровождают разные пассажиры, контролеры, торговцы и музыканты. Мобильная сеть предательски исчезает, расписание подводит, одни ругаются, а кто-то разговаривает по видеосвязи, песни, стихи, полезные товары и даже философия – всё это часть обстановки, воссозданная игровым театром».
*   «В закопчённое окно можно постоянно наблюдать настоящий флаг России, с тремя горизонтальными полосами – серой, чёрной и белой. Небо, деревья и снег – больше ничего».
*   «Алексей Золотовицкий — мастер по рассыпанию комических моментов даже по самым трагическим театральным полотнам».
*   «Потому я успела и похихикать, и поностальгировать, и посоглашаться или внутренне поспорить с философскими измышлениями, звучащими со сцены… ну, и поплакать, конечно».